## Data exploration

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
news_df = pd.read_csv("data/Sora_LREC2020_biasedsentences.csv")

Seems that each instance consists of an article with sentences and a bias score between 1 and 4 is assigned to the article as a whole, the title, and each sentence.

In [2]:
news_test_sentence = news_df.iloc[0] # visual inspection
print(f"sentence: {news_test_sentence['s0']}", f"score: {news_test_sentence['0']}")
print(f"dataset score range: {news_df['article_bias'].min(),news_df['article_bias'].max()}")
# pd.DataFrame(news_test_sentence)

sentence: [0]: LOUISVILLE - Dan Johnson posted a final message on Facebook to his friends and family on Wednesday afternoon. score: 3
dataset score range: (np.int64(1), np.int64(4))


In [3]:
# model source: https://huggingface.co/cardiffnlp/twitter-roberta-base-2022-
# adapter source: https://huggingface.co/SOUMYADEEPSAR/text_level_bias_roberta-twitter
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from adapters import AutoAdapterModel
tokeniser = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
model = AutoAdapterModel.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
adapter = model.load_adapter("SOUMYADEEPSAR/text_level_bias1", set_active=True)
classifier = pipeline('text-classification', model=model, tokenizer=tokeniser) # cuda = 0,1 based on gpu availability

RobertaAdapterModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['hea

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\spiro\LifeHub\~Education\~Leiden\MSc\Text mining\Assignments\venv\Lib\site-packages\adapters\loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dic

In [10]:
#preprocess news data
def extract_sentences_and_labels(df):
    sentences_list = []
    labels_list = []
    for _, row in df.iterrows():
        for i in range(0, 20):  # Sentences are named f"s0" to f"s19"
            if type(row[f"s{i}"]) == str: # skipping nan sentences
                sentences_list.append(row[f"s{i}"])
                labels_list.append(labels_to_binary(row[f"{i}"]))
            
    return sentences_list, labels_list

sentences, labels = extract_sentences_and_labels(news_df)
baseline_tokeniser = tokeniser
inputs = baseline_tokeniser(sentences)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs["input_ids"], labels, test_size = 0.1, shuffle = True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, shuffle = True, random_state=42)

In [4]:
def labels_to_binary(label: int):
    return 0 if label == 1 or label == 2 else 1

# comparing bias score obtained from classifier to true bias score for a test sentence from the news dataset
test_sent_pred = classifier(news_test_sentence["s0"])
test_sent_pred = test_sent_pred[-1]['label']
test_sent_y = labels_to_binary(news_test_sentence["0"])
print(f"pred: {test_sent_pred}, y: {test_sent_y}")

pred: 0, y: 1


In [6]:
test = False
def eval_classifier(df, classifier):
    pred = []
    y = []
    count = 0
    for row_idx,row in tqdm(df.iterrows()):
        if test and row_idx < 20:
            continue
        if test and row_idx == 30:
            break
        for sent_idx in range(20):
            try:
                pred.append(classifier(row[f"s{sent_idx}"])[-1]["label"])
            except ValueError:
                # Todo: add if statement checking if nan
                pred.append(0) # seems (?) to be the case that for NaNs, minimum bias is assigned
            y.append(labels_to_binary(row[f"{sent_idx}"]))
    return pred, y
pred, y = eval_classifier(news_df, classifier)

1it [00:03,  3.20s/it]


KeyboardInterrupt: 

In [ ]:
test = False
def eval_classifier(df, classifier):
    pred = []
    y = []
    count = 0
    for row_idx,row in tqdm(df.iterrows()):
        if test and row_idx < 20:
            continue
        if test and row_idx == 30:
            break
        for sent_idx in range(20):
            try:
                pred.append(classifier(row[f"s{sent_idx}"])[-1]["label"])
            except ValueError:
                # Todo: add if statement checking if nan
                pred.append(0) # seems (?) to be the case that for NaNs, minimum bias is assigned
            y.append(labels_to_binary(row[f"{sent_idx}"]))
    return pred, y
pred, y = eval_classifier(news_df, classifier)

In [21]:
def calc_accuracy(pred, y):
    pred_copy = [np.int64(label) for label in pred]
    y_copy = [np.int64(label) for label in y]
    count = 0
    for index,label in enumerate(pred_copy):
        if y_copy[index] == label:
            count += 1
    return count/len(pred_copy)
# calc_accuracy(pred, y)

## Fine tune classifier and compare to baseline

In [23]:
from datasets import Dataset
train_dataset = Dataset.from_dict({
    'input_ids': X_train,
    'label': y_train
})

eval_dataset = Dataset.from_dict({
    'input_ids':X_val,
    'label': y_val
})

test_dataset = Dataset.from_dict({
    'input_ids':X_test,
    'label':y_test
})

In [12]:
def compute_metrics(pred, y):
    return calc_accuracy(pred,y)

In [25]:
from transformers import TrainingArguments
from transformers import Trainer
test = True
args = TrainingArguments(
    "baseline-bias-classifier",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset=train_dataset,
    eval_dataset= eval_dataset,
    tokenizer= baseline_tokeniser
)

In [28]:
# this seems like it will take very long (3hrs or so). Maybe we should get this to work with GPUs. I think that would involve using a data collator so that everything is the same shape.

# fine-tuning
trainer.train()

  0%|          | 0/1194 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

{'eval_runtime': 26.7795, 'eval_samples_per_second': 13.219, 'eval_steps_per_second': 1.68, 'epoch': 1.0}
{'loss': 0.6684, 'grad_norm': 5.961797714233398, 'learning_rate': 1.1624790619765495e-05, 'epoch': 1.26}


  0%|          | 0/45 [00:00<?, ?it/s]

{'eval_runtime': 27.7307, 'eval_samples_per_second': 12.766, 'eval_steps_per_second': 1.623, 'epoch': 2.0}
{'loss': 0.6364, 'grad_norm': 5.191866874694824, 'learning_rate': 3.2495812395309884e-06, 'epoch': 2.51}


  0%|          | 0/45 [00:00<?, ?it/s]

{'eval_runtime': 31.2682, 'eval_samples_per_second': 11.321, 'eval_steps_per_second': 1.439, 'epoch': 3.0}
{'train_runtime': 2928.2089, 'train_samples_per_second': 3.256, 'train_steps_per_second': 0.408, 'train_loss': 0.6480254168486476, 'epoch': 3.0}


TrainOutput(global_step=1194, training_loss=0.6480254168486476, metrics={'train_runtime': 2928.2089, 'train_samples_per_second': 3.256, 'train_steps_per_second': 0.408, 'total_flos': 276629152145364.0, 'train_loss': 0.6480254168486476, 'epoch': 3.0})

In [29]:
# obtaining predictions from fine-tuned model as logits
predictions = trainer.predict(test_dataset)

  0%|          | 0/50 [00:00<?, ?it/s]

In [30]:
predictions

PredictionOutput(predictions=(array([0.7038864 , 0.6937561 , 0.6639484 , 0.7868625 , 0.48734406,
       0.9180164 , 0.46692032, 0.61046344, 0.73482084, 0.92367446,
       0.67665315, 0.40204564, 0.5182122 , 0.61428994, 0.90572333,
       0.5524112 , 0.5988605 , 0.6766929 , 0.5960391 , 0.80469024,
       0.7320262 , 0.7888113 , 0.8483089 , 0.75849587, 0.73935497,
       0.6702986 , 0.7388642 , 0.72211707, 0.7461526 , 0.5389875 ,
       0.96301246, 0.418186  , 0.99670947, 0.86583173, 0.3600293 ,
       0.85527205, 0.63864124, 0.81468713, 0.8616187 , 0.83753294,
       0.6972611 , 0.5107569 , 0.7223513 , 0.55591244, 0.84153444,
       0.97584426, 0.6572989 , 0.70701265, 0.6325366 , 1.5262616 ],
      dtype=float32), array([[ 0.5572506 , -0.5410291 ],
       [-0.36843497,  0.3525593 ],
       [ 0.86653364, -0.8231727 ],
       [ 0.47954106, -0.45747066],
       [-0.42672837,  0.4048805 ],
       [ 0.29323417, -0.2729563 ],
       [ 0.39285076, -0.38551953],
       [-0.22053103,  0.2031196 

In [32]:
from scipy.special import softmax

# converting logits into predicted labels
logits = np.array(predictions[0][-1])
probabilities = softmax(logits, axis=1)
predicted_classes = np.argmax(probabilities, axis = 1)
print(predicted_classes)
test_dataset["label"]

compute_metrics(predicted_classes, test_dataset["label"])

[0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0
 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0]


0.5725190839694656

## Make predictions for X notes data

In [ ]:
...

## Compare classifier predictions to human predictions

In [ ]:
# my idea for how to evaluate:
# 1) we set some guidelines for how we will evaluate notes 
# 2) we individually make labels for one days worth of notes (that's about 200 notes; this would be the train set)
# 3) we calculate out interrater agreement (if it's very bad, we revise our guidelines and re-label)
# 4) we average our labels and use that to further fine-tune the classifier
# 5) we repeat step 2 on a new set of notes (this would be the test set)
# 6) we make predictions using the classifier obtained from step 4
# 7) either we calculate the MSE using our labels as ground truth, or we calculate three interrater agreements (chico-andrew, chico-classifier, andrew-classifier)
# and see if the human-human agreement is better than the human-classifier agreement.
...